In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [3]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [4]:
channel = channel_list[0]
rank = channel.select_one('.rank').get_text().strip()
rank = int(rank)
rank

1

In [7]:
category = channel.select_one('p.category').get_text().strip()
category[1:-1]

'음악/댄스/가수'

In [11]:
name = channel.select_one('.subject a').get_text().strip()
name

'BLACKPINK'

In [12]:
subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
subscriber

'7020만'

In [13]:
view = channel.select_one('.view_cnt').get_text().strip()
video = channel.select_one('.video_cnt').get_text().strip()

view, video

('214억5709만', '395개')

- 반복문으로 페이지내에 있는 데이터 수집하기

In [14]:
lines = []

for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()
    lines.append([rank,category[1:-1],name,subscriber,view,video])
    

In [16]:
df = pd.DataFrame(lines, columns=['순위','카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,7020만,214억5709만,395개
1,2,음악/댄스/가수,HYBE LABELS,6300만,204억9684만,725개
2,3,음악/댄스/가수,BANGTANTV,6150만,140억7794만,"1,646개"
3,4,음악/댄스/가수,SMTOWN,2930만,230억1641만,"3,779개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개


- 숫자로 변환하는 함수

In [17]:
subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
subscriber

'384만'

In [18]:
subscriber.replace('만', '0000')

'3840000'

In [39]:
def convert_unit(s):
    s = s.replace('만', '0000').replace('억','').replace('개','').replace(',','')
    return f'{int(s):,d}'

In [41]:
df['구독자수'] = df['구독자수'].apply(convert_unit)
df['조회수'] = df['조회수'].apply(convert_unit)
df['비디오수'] = df['비디오수'].apply(convert_unit)
df.head()

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"70,200,000","21,457,090,000",395
1,2,음악/댄스/가수,HYBE LABELS,"63,000,000","20,496,840,000",725
2,3,음악/댄스/가수,BANGTANTV,"61,500,000","14,077,940,000","1,646"
3,4,음악/댄스/가수,SMTOWN,"29,300,000","23,016,410,000","3,779"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


- 두번째 페이지 

In [43]:
#Xpath로 찾아가기 - //*[@id="list-skin"]/nav/span/a[1]

driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
time.sleep(2)

In [44]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [45]:
lines = []

for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()
    lines.append([rank,category[1:-1],name,convert_unit(subscriber),convert_unit(view),convert_unit(video)])


In [46]:
df = pd.DataFrame(lines, columns=['순위','카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,순위,카테고리,채널명,구독자수,조회수,비디오수
0,101,TV/방송,워크맨-Workman,"3,790,000","743,430,000",154
1,103,애완/반려동물,크림히어로즈,"3,780,000","1,415,780,000","1,011"
2,102,음악/댄스/가수,WINNER,"3,780,000","1,204,270,000",446
3,104,키즈/어린이,Lime Tube[라임튜브],"3,740,000","2,646,690,000","1,808"
4,105,BJ/인물/연예인,waveya 2011,"3,730,000","1,356,390,000",710


- page 1~10 크롤링

In [47]:
lines = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        rank = int(channel.select_one('.rank').get_text().strip())
        category = channel.select_one('p.category').get_text().strip()
        name = channel.select_one('.subject a').get_text().strip()
        subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
        view = channel.select_one('.view_cnt').get_text().strip()
        video = channel.select_one('.video_cnt').get_text().strip()
        lines.append([rank,category[1:-1],name,convert_unit(subscriber),convert_unit(view),convert_unit(video)])


In [49]:
df = pd.DataFrame(lines, columns=['순위','카테고리','채널명','구독자수','조회수','비디오수'])
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오수
995,996,음악/댄스/가수,Reynah,"550,000","148,880,000",246
996,997,TV/방송,J.Y. Park,"550,000","93,730,000",90
997,998,게임,[ALTUBE] 김성현TV,"550,000","422,100,000","2,121"
998,1000,게임,방앗간 비둘기,"550,000","356,900,000","4,381"
999,999,게임,빅헤드,"550,000","326,600,000","3,540"


In [50]:
df.to_csv('유튜브랭킹.csv', index=False)

In [51]:
driver.close()